# scraper Clasificados de los tiempos

notebook para obtenere datos de la página de los clasificados de los tiempos
con beautifulsoup

LINK: https://clasificados.lostiempos.com/inmuebles 

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import math
from datetime import datetime
from pyquery import PyQuery as pq
from bs4 import BeautifulSoup


In [3]:
BASE_URL = "https://clasificados.lostiempos.com"
URL = f"{BASE_URL}/inmuebles?sort_by=created&sort_order=DESC&page="

HEADERS = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50"
                         ".0.2661.102 Safari/537.36"}
posts = []

MONTHS_SPANISH = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]

response = requests.get(f"{URL}{0}", headers=HEADERS)

soup = BeautifulSoup(response.content, 'html.parser')

In [42]:
str(soup.find_all(class_="views-row")[0])

'<div class="views-row views-row-1 24 Julio 2023">\n<div class="title">\n<span class="field-content"><a href="/anuncio/inmuebles/venta-de-terreno-sup/2021653">Venta De Terreno Sup</a></span></div>\n<div class="publish-date">Publicado: <span class="field-content">24 Julio 2023</span></div>\n<div class="body">\n<span class="field-content">VENTA DE TERRENO Sup. 905.56 m2 ubicación distrito 4 Otb Noroeste zona radio urbano calle Cullavi Ref. 63872948\n</span></div>\n<div class="ads-price">\n<span class="field-content"></span> </div>\n<div class="description">\n<span class="field-content">Cochabamba</span> -  <span class="field-content">Venta</span> -  <span class="field-content">Lote / Terreno</span>\n</div>\n<div class="buttons">\n<div class="contact-button">\n<span class="field-content"><a href="/anuncio/inmuebles/venta-de-terreno-sup/2021653">Contactar</a></span> </div>\n<div class="sharethis-buttons">\n</div>\n</div>\n</div>'

In [26]:
def append_elements(doc, posts):
    l = soup.find_all(class_='views-row')
    for i, element in enumerate(l):
        data = get_data(element)
        posts.append(data)

In [24]:
# clean the text
def clean_text(s):
    return s.replace('\n', '').strip()

# get data from text like 'Publicado: 23 Marzo 2023'
def get_date(s):
    original_date = s.replace('Publicado: ', '')
    dt = original_date.split(' ')
    return f"{dt[2]}-{str(MONTHS_SPANISH.index(dt[1])).zfill(2)}-{dt[0].zfill(2)}"

# Get celphone and phone number 
def get_phones(s):
    return re.findall(r"([\d]{8}|[\d]{7})", s)

def get_price(s):
    # TODO: hacer el calculo de precios.
    pass

In [54]:
def get_data(element):
    title = element.find(class_='title').text
    date = element.find(class_='publish-date').text
    description = element.find(class_='body').text
    price = element.find(class_='ads-price').text
    keys = element.find(class_='description').text
    path = element.find(class_='title').find('a')['href']
    
    if '...' in description:
        description = get_data_from_page(path)

    return {
        'title': clean_text(title),
        'date': get_date(clean_text(date)),
        'description': clean_text(description),
        'price': clean_text(price),
        'keys': clean_text(keys),
        'path': clean_text(path),
        'html': str(element),
    }

# get_data(doc.find('.views-row').eq(0))

In [52]:
def get_data_from_page(path):
    response = requests.get(f"{BASE_URL}{path}", headers=HEADERS)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.find(class_="field-items").text

s = get_data_from_page('/anuncio/inmuebles/propietario-3-dormitorios-uno/2022644')

In [53]:
s

'PROPIETARIO 3 dormitorios, (uno suite), 134 m2, dependencias empleada, parqueo, piscina atemperada, parrilleros, quinto piso, edificio Onix, batallón Colorados 2363 casi Melchor Pérez de Holguín, Bs. 2.400, facturado, no incluye expensas (200 Bs), garantía 5.000 Bs. (anticrético 320.000 Bs.). Ref. Telf. 71723994.\n'

In [47]:
# TEST
n = 2
print("get data from:", f"{URL}{n}")
response = requests.get(f"{URL}{n}", headers=HEADERS)
# doc = pq(response.content)
soup = BeautifulSoup(response.content, 'html.parser')
test_soup = []
append_elements(soup, test_soup)

get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=2


In [55]:
test_soup

[{'title': 'Bs 1 200 Casa',
  'date': '2023-06-28',
  'description': 'BS 1.200  Casa independiente. Con patio amplio, garaje, 2 cuartos, cocina, baño. whatsapp 44291080',
  'price': '$us 150',
  'keys': 'Cochabamba -  Alquiler -  Casa / Chalet',
  'path': '/anuncio/inmuebles/bs-1-200-casa/2022849'},
 {'title': 'Oficina Céntrica En Alquiler',
  'date': '2023-06-27',
  'description': 'OFICINA céntrica en alquiler. 70774733.',
  'price': '',
  'keys': 'Cochabamba -  Alquiler -  Oficina',
  'path': '/anuncio/inmuebles/oficina-centrica-en-alquiler/2021814'},
 {'title': 'En Venta Casa Zona',
  'date': '2023-06-27',
  'description': 'EN VENTA Casa zona campo ferial llamar: 68103436, 71190552',
  'price': '',
  'keys': 'Cochabamba -  Venta -  Casa / Chalet',
  'path': '/anuncio/inmuebles/en-venta-casa-zona/2022252'},
 {'title': 'Lotes En Venta Sup',
  'date': '2023-06-27',
  'description': 'LOTES en venta, Sup. 310 m2, San Benito, carretera antigua, Santa Cruz, Km 39. 65363254.',
  'price': ''

In [35]:
number_elements = int(re.findall(r'\d+', soup.find(class_='resume-search').text)[0])
NUMBER_OF_PAGES = math.ceil(number_elements / 20)

for n in range(1, NUMBER_OF_PAGES):
    print("get data from:", f"{URL}{n}")
    response = requests.get(f"{URL}{n}", headers=HEADERS)
    # doc = pq(response.content)
    soup = BeautifulSoup(response.content, 'html.parser')
    append_elements(soup, posts)

print("total de publicaciones:", len(posts))

get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=1
get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=2
get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=3
get data from: https://clasificados.lostiempos.com/inmuebles?sort_by=created&sort_order=DESC&page=4
total de publicaciones: 144


In [36]:
df = pd.DataFrame(posts)
df

,title,date,description,price,keys,path
0,Vendo Casa En Argentina,2023-06-27,"VENDO Casa en Argentina, ciudad de Salta, 49.0...",$us 49 000,Cochabamba - Venta - Casa / Chalet,/anuncio/inmuebles/vendo-casa-en-argentina/202...
1,Departamento En Alquiler Cuenta,2023-06-27,"DEPARTAMENTO en alquiler, cuenta: 1 suite, 2 d...",,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/departamento-en-alquiler-cu...
2,Lotes En Venta Sup,2023-06-27,"LOTES en venta, Sup. 310 m2, San Benito, carre...",,Cochabamba - Venta - Lote / Terreno,/anuncio/inmuebles/lotes-en-venta-sup/2022264
3,En Venta Casa Zona,2023-06-27,EN VENTA Casa zona campo ferial llamar: 681034...,,Cochabamba - Venta - Casa / Chalet,/anuncio/inmuebles/en-venta-casa-zona/2022252
4,1 400 Bs Departamento,2023-06-27,"1.400 BS. departamento, 2 dormitorios, roperos...",,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/1-400-bs-departamento/2022267
...,...,...,...,...,...,...
139,Vendo Departamento A Estrenar,2023-06-24,"VENDO departamento a estrenar, Fidel Anze, 3 d...",,Cochabamba - Venta - Departamento,/anuncio/inmuebles/vendo-departamento-estrenar...
140,Venta De Terreno Sup,2023-06-24,VENTA DE TERRENO Sup. 905.56 m2 ubicación dist...,,Cochabamba - Venta - Lote / Terreno,/anuncio/inmuebles/venta-de-terreno-sup/2021653
141,Alquilo Casa Independiente Con,2023-06-24,"ALQUILO casa independiente, con todas las como...",,Cochabamba - Alquiler - Casa / Chalet,/anuncio/inmuebles/alquilo-casa-independiente-...
142,Alquilo Departamento 3 Dormitorios,2023-06-24,"ALQUILO departamento 3 dormitorios, calle Bapt...",,Cochabamba - Alquiler - Departamento,/anuncio/inmuebles/alquilo-departamento-3-dorm...


In [11]:
# Save dataframe as .csv file 
df.to_csv(f'./csv/los_tiempos_clasificados_{datetime.now(timezone('US/Eastern')).isoformat()[:10]}.csv', sep=';')

In [39]:
from pytz import timezone
datetime.now(timezone('US/Eastern')).isoformat()[:10]

'2023-07-27'

In [44]:
datestamp = "2023-06-27"
posts_filtered = [d for d in posts if d['date'] == datestamp]
print(len(posts))
print(len(posts_filtered))

144
40
